In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
pyo.init_notebook_mode()

Output hidden; open in https://colab.research.google.com to view.

In [3]:
df = pd.read_csv("Environment_Temperature_change_E_All_Data_NOFLAG.csv", encoding='latin-1') # csv file is encoding as latin-1 type
df_countrycode=pd.read_csv('FAOSTAT_data_11-24-2020.csv') #this csv file includes ISO-3 Country Code, this mentioned in Data Wrangling 

display(df.head(5))
print('\n')

print('\n')
print("Months")
display(df.Months.unique())
print('\n')
print("Elements")
display(df.Element.unique())
print('\n')
display(df_countrycode.head(5))

,Area Code,Area,Months Code,Months,Element Code,Element,Unit,Y1961,Y1962,Y1963,...,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
0,2,Afghanistan,7001,January,7271,Temperature change,°C,0.777,0.062,2.744,...,3.601,1.179,-0.583,1.233,1.755,1.943,3.416,1.201,1.996,2.951
1,2,Afghanistan,7001,January,6078,Standard Deviation,°C,1.950,1.950,1.950,...,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950
2,2,Afghanistan,7002,February,7271,Temperature change,°C,-1.743,2.465,3.919,...,1.212,0.321,-3.201,1.494,-3.187,2.699,2.251,-0.323,2.705,0.086
3,2,Afghanistan,7002,February,6078,Standard Deviation,°C,2.597,2.597,2.597,...,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597
4,2,Afghanistan,7003,March,7271,Temperature change,°C,0.516,1.336,0.403,...,3.390,0.748,-0.527,2.246,-0.076,-0.497,2.296,0.834,4.418,0.234






Months


array(['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August', 'September', 'October', 'November', 'December',
       'Dec\x96Jan\x96Feb', 'Mar\x96Apr\x96May', 'Jun\x96Jul\x96Aug',
       'Sep\x96Oct\x96Nov', 'Meteorological year'], dtype=object)



Elements


array(['Temperature change', 'Standard Deviation'], dtype=object)

,Country Code,Country,M49 Code,ISO2 Code,ISO3 Code,Start Year,End Year
0,2,Afghanistan,4.0,AF,AFG,NaN,NaN
1,5100,Africa,2.0,NaN,X06,NaN,NaN
2,284,Åland Islands,248.0,NaN,ALA,NaN,NaN
3,3,Albania,8.0,AL,ALB,NaN,NaN
4,4,Algeria,12.0,DZ,DZA,NaN,NaN


In [4]:
df.rename(columns = {'Area':'Country Name'},inplace = True)
df.set_index('Months', inplace=True)
df.rename({'Dec\x96Jan\x96Feb': 'Winter', 'Mar\x96Apr\x96May': 'Spring', 'Jun\x96Jul\x96Aug':'Summer','Sep\x96Oct\x96Nov':'Fall'}, axis='index',inplace = True)
df.reset_index(inplace = True)

df = df[df['Element'] == 'Temperature change']

df_countrycode.drop(['Country Code','M49 Code','ISO2 Code','Start Year','End Year'],axis=1,inplace=True)
df_countrycode.rename(columns = {'Country':'Country Name','ISO3 Code':'Country Code'},inplace=True)

df = pd.merge(df, df_countrycode, how='outer', on='Country Name')

df.drop(['Area Code','Months Code','Element Code','Element','Unit'],axis=1,inplace=True)

df = df.melt(id_vars=["Country Code", "Country Name","Months",], var_name="year", value_name="tem_change")
df["year"] = [i.split("Y")[-1] for i in df.year]

display(df.sample(5))


,Country Code,Country Name,Months,year,tem_change
105512,WSM,Samoa,September,1982,0.066
180797,BDI,Burundi,October,1998,NaN
97030,NaN,South-Eastern Asia,November,1980,0.304
188028,PHL,Philippines,July,1999,0.288
127187,BGR,Bulgaria,Summer,1987,0.652


In [5]:
df_c =df.copy()
df_c.set_index("year", inplace=True)
df_c = df_c.loc[['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']]
df_c.reset_index(inplace = True)


df_c = df_c.groupby(
   ['Country Name',]
).agg(
    {
        'tem_change':'mean', 
        
    }
)
df_c.reset_index(inplace = True)
df_c = df_c.sort_values(by=['tem_change'],ascending=False).head(10)

fig = px.bar(df_c, x="Country Name", y='tem_change' ,text='tem_change', 
             title="Top ten countries that have highest temperature change in the last decades",
             color_discrete_sequence=['green'])

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')



fig.update_layout(
    autosize=False,
    width=1000,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
  
    template='seaborn',
    paper_bgcolor="rgb(234, 234, 242)",
    legend=dict(
        orientation="v",
        yanchor="bottom",
        y=0.3,
        xanchor="left",
        x=1.02
))
fig.update_xaxes( tickangle = 10,
        title_text = "Countries",
        title_font = {"size": 15},
        title_standoff = 0)
fig.update_yaxes(showticklabels=False,tickmode="auto", title='Temperature Change',title_standoff = 0)

fig.show()

In [6]:
import plotly.express as px

df_c = df.copy()
df_c.set_index("year", inplace=True)
df_c = df_c.loc[['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']]
df_c.reset_index(inplace=True)

df_c = df_c.groupby(['Country Name']).agg({'tem_change': 'mean'})
df_c.reset_index(inplace=True)
df_c = df_c.sort_values(by=['tem_change'], ascending=True).head(10)

fig = px.bar(df_c, x="Country Name", y='tem_change', text='tem_change', color='tem_change',
             title="Top ten countries that have lowest temperature change in the last decades")
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')

fig.update_layout(
    autosize=False,
    width=1000,
    height=600,
    margin=dict(l=50, r=50, b=100, t=100, pad=4),
    template='seaborn',
    paper_bgcolor="rgb(234, 234, 242)",
    legend=dict(orientation="v", yanchor="bottom", y=0.3, xanchor="left", x=1.02))

fig.update_xaxes(
    tickangle=10,
    title_text="Countries",
    title_font={"size": 15},
    title_standoff=0)

fig.update_yaxes(showticklabels=False, title='Temperature Change')

fig.show()

In [ ]:
df0 = df[df['Months'] == 'Meteorological year'] # new data frame includes only yearly values
df1 = df0[df0['Country Name'] == 'World'] # from new data frame filtering World's data
df2 = df0[df0['Country Name'] == 'Annex I countries'] # from new data frame filtering'Annex I countries' data
df3 = df0[df0['Country Name'] == 'Non-Annex I countries'] # from new data frame filtering  'Non-Annex I countries' data



fig = go.Figure()


fig.add_trace(go.Scatter(x = df1.year, y=df1.tem_change,
                    mode='markers',
                    name='World',
                    marker=dict(color='red')))

fig.add_trace(go.Scatter(x = df2.year , y=df2.tem_change,
                    mode='lines',
                    name='Annex I countries',
                    line=dict(color='green')))


fig.add_annotation(x='55',y=2.098,
                   xref="x", yref="y",
            text="The hottest record",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=20,
        ay=-30,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#ff7f0e",
        opacity=0.8
        )
fig.add_trace(go.Scatter(x = df3.year , y=df3.tem_change,
                    mode='lines', name='Non-Annex I countries'))



fig.update_layout(
    autosize=False,
    width=1000,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    
    template='seaborn',
    paper_bgcolor="rgb(234, 234, 242)",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
))

fig.update_xaxes(type='category',title='Years')
fig.update_yaxes(title='Temperature Change')

fig.show()

In [ ]:
df0 = df[df['Country Name'] == 'World']
df1 = df0[df0['Months'] == 'Winter']
df2 = df0[df0['Months'] == 'Spring']
df3 = df0[df0['Months'] == 'Summer']
df4 = df0[df0['Months'] == 'Fall']
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df1['year'], y=df1.tem_change,
                    mode='lines',
                    name='Winter',
                    line_color='blue'))
fig.add_trace(go.Scatter(x=df2['year'], y=df2.tem_change,
                    mode='markers',
                    name='Spring',
                    marker_color='green'))
fig.add_trace(go.Scatter(x=df3['year'], y=df3.tem_change,
                    mode='lines',
                    name='Summer',
                    line_color='red'))
fig.add_trace(go.Scatter(x=df4['year'], y=df4.tem_change,
                    mode='markers',
                    name='Fall',
                    marker_color='orange'))

fig.add_annotation(x='55',y=2.165,
                   xref="x", yref="y",
            text="The hottest winter",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=20,
        ay=-30,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#ff7f0e",
        opacity=0.8
        )
fig.update_layout(
    autosize=False,
    width=1000,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    template='seaborn',
    paper_bgcolor="rgb(234, 234, 242)",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
))
fig.update_xaxes(type='category',title='Years')
fig.update_yaxes(title='Temperature Change')

fig.show()

In [ ]:
import warnings
warnings.filterwarnings('ignore')
df0 = df[df['Country Name'] == 'World']
df0.set_index("Months", inplace=True)
df0 = df0.loc[['January', 'February', 'March', 'April', 'May', 'June', 'July','August', 'September', 'October', 'November', 'December' ]]
df0.reset_index(inplace = True)


fig = px.line_polar(df0, r=df0.tem_change, theta=df0.Months, animation_frame='year', line_close=True, color_discrete_sequence=['red'])
fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[-0.5, 3]
    )),
    autosize=False,
    width=1000,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    template='seaborn',
    paper_bgcolor="rgb(234, 234, 242)",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
))


fig.show()

In [ ]:
df_tem_change = df.copy() 
df_tem_change = df_tem_change[df_tem_change['Months'] == 'Meteorological year'] 
df_tem_change.drop(['Months'],axis=1,inplace=True) 
df_tem_change.to_csv(r'./Temperature_change_Data.csv',index=False)

In [ ]:
df_map = df.copy() 

df_map = df_map[df_map['Months'] == 'Meteorological year'] 
df_map['°C'] = ['<=-1.5' if x<=(-1.5) else '<=-1.0' if (-1.5)<x<=(-1.0) else '<=0.0' if (-1.0)<x<=0.0  else '<=0.5' if 0.0<x<=0.5 else '<=1.5' if 0.5<x<=1.5 else '>1.5' if 1.5<=x<10 else 'None' for x in df_map['tem_change']]

In [ ]:
fig = px.choropleth(df_map, 
                    locations="Country Code",
                    color="tem_change", 
                    locationmode='ISO-3',
                    hover_name="Country Name",
                    hover_data=['tem_change'],
                    animation_frame=df_map.year,
                    labels={'tem_change':'The Temperature Change', '°C':'°C'},
                    category_orders={'tem_change':['<=-1.5','<=-1.0','<=0.0','<=0.5','<=1.5','>1.5','None']},
                    color_discrete_map={'<=-1.5':"#005824",
                                        '<=-1.0':"#238b45",
                                        '<=0.0':"#41ae76",
                                        '<=0.5':"#66c2a4",
                                        '<=1.5':"#99d8c9",
                                        '>1.5':"#ccece6",
                                        'None':"#f7fcf5"},
                    title='Temperature Change - 1961 - 2019')

fig.update_layout(
    autosize=False,
    width=1200,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    template='seaborn',
    paper_bgcolor="rgb(234, 234, 242)",
    legend=dict(
        orientation="v",
        yanchor="auto",
        y=1.02,
        xanchor="right",
        x=1
    ))

fig.show()